In [ ]:
%matplotlib inline
from pathlib import Path
import numpy as np
import scipy
import matplotlib.pyplot as plt
import sklearn
import IPython.display as ipd
import librosa, librosa.display

import warnings
warnings.filterwarnings('ignore')

# Basic Feature Extraction
## Extração básica de recursos

De alguma forma, devemos extrair as características do nosso sinal de áudio que são mais relevantes para o problema que estamos tentando resolver. 

Por exemplo, se quisermos classificar instrumentos por **timbre**, queremos características que possam nos ajudar a destinguir sons por seu **timbre** e não por seu **tom**.

> Este processo é conhecido como `extração de recursos` (feature extraction).

Para esta tarefa vamos analisar vinte arquivos de áudio: 

* dez amostras de **`kick drum`** ou **bumbo** 
* dez amostras de **`snare drum`** ou **caixa**

<div align="center" style="width: 100%;">
    <img src="imgs/drum-set-labelled.jpg" style="width: 50%;">
</div>

In [ ]:
kick_signals = [ librosa.load(p)[0] for p in Path().glob('audios/drum_samples/train/kick_*.mp3') ]
snare_signals = [ librosa.load(p)[0] for p in Path().glob('audios/drum_samples/train/snare_*.mp3') ]

In [ ]:
print(f'Quantidade de kick drums: {len(kick_signals)}')
print(f'Quantidade de snare drums: {len(snare_signals)}')

#### Kick drum signals:

In [ ]:
plt.figure(figsize=(18, 8))

for i, x in enumerate(kick_signals):
    plt.subplot(2, 5, i+1)
    librosa.display.waveplot(x[:10000], alpha=0.8)
    plt.ylim(-1, 1)

#### Snare drum signals

In [ ]:
plt.figure(figsize=(18, 8))

for i, x in enumerate(snare_signals):
    plt.subplot(2, 5, i+1)
    librosa.display.waveplot(x[:10000], alpha=0.8)
    plt.ylim(-1, 1)

## Create a Feature Vector

Vamos criar um vetor de recursos (Feature Vector), para armazenar nossa coleção de recursos. 

Abaixo temos o método `extract_features`, que é uma função simples para construção de um vetor de recursos bidimensionais a partir de um determinado sinal:

In [ ]:
def extract_features(signal):
    return [
        librosa.feature.zero_crossing_rate(signal)[0, 0],
        librosa.feature.spectral_centroid(signal)[0, 0],
    ]

In [ ]:
# Usando list comprehension para agrupar os vetores

kick_features = np.array([extract_features(x) for x in kick_signals])
snare_features = np.array([extract_features(x) for x in snare_signals])

#### Plot do histograma das características de cada uma das classes:

In [ ]:
plt.figure(figsize=(16, 6))

plt.hist(kick_features[:,0], color='b', range=(0, 0.2), alpha=0.5, bins=20)
plt.hist(snare_features[:,0], color='r', range=(0, 0.2), alpha=0.5, bins=20)
plt.legend(('kicks', 'snares'))
plt.xlabel('Zero Crossing Rate')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(16, 6))

plt.hist(kick_features[:,1], color='b', range=(0, 4000), bins=30, alpha=0.6)
plt.hist(snare_features[:,1], color='r', range=(0, 4000), bins=30, alpha=0.6)
plt.legend(('kicks', 'snares'))
plt.xlabel('Spectral Centroid (frequency bin)')
plt.ylabel('Count')
plt.show()

## Feature Scaling
### Dimensionamento de recursos

As características que usamos no exemplo anterior incluíam a taxa de zero-crossing (zero crossing rate) e o centroide espectral (spectral centroid). Esses dois recursos são expressos usando unidades diferentes. Essa discrepância pode gerar problemas ao realizar uma classificação a posteriori. Portanto, normalizaremos cada vetor de recursos em uma faixa comum, armazenando também os parâmetros desta normalização para uso futuro.

Neste caso usaremos o `sklearn.preprocessing.MinMaxScaler`, que retorna uma série de valores escalonados de tal forma que cada dimensão do recurso está na faixa de -1 a 1.

Vamos concatenar todos os vetores de recursos em uma tabela de recursos:

In [ ]:
feature_table = np.vstack((kick_features, snare_features))

print(feature_table.shape)

Escalar cada dimensão de recurso para a faixa entre -1 e 1:

In [ ]:
scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-1, 1))
training_features = scaler.fit_transform(feature_table)

print(training_features.min(axis=0))
print(training_features.max(axis=0))

Plot das características dimensionadas:

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(training_features[:10,0], training_features[:10,1], c='b')
plt.scatter(training_features[10:,0], training_features[10:,1], c='r')
plt.xlabel('Zero Crossing Rate')
plt.ylabel('Spectral Centroid')
plt.show()

# Energy and RMSE

A energia ([Wikipedia](https://en.wikipedia.org/wiki/Energy_(signal_processing%29); FMP, p. 66) de um sinal corresponde à magntiude total do sinal. Para sinais de áudio, isso corresponde aproximadamente à intensidade do sinal. A energia em um sinal é definida como:

$$\Large \sum_n \left| x(n) \right|^2$$
 
A energia da raíz quadrática média, do inglês root-mean-square energy (RMSE), é definida por:

$$\Large \sqrt{ \frac{1}{N} \sum_n \left| x(n) \right|^2 }$$

In [ ]:
x, sr = librosa.load('audios/simple_loop.wav')

In [ ]:
librosa.get_duration(x, sr)

In [ ]:
ipd.Audio('audios/simple_loop.wav')

In [ ]:
plt.figure(figsize=(16, 6))
librosa.display.waveplot(x, sr=sr, alpha=0.8)
plt.show()

Computar o short-time energy usando apenas list comprehension:

In [ ]:
hop_length = 256
frame_length = 512

energy = np.array([
    sum(abs(x[i:i+frame_length]**2))
    for i in range(0, len(x), hop_length)
])

print(energy.shape)

Calcular o RMSE usando a função [librosa.feature.rms](https://librosa.github.io/librosa/generated/librosa.feature.rms.html):

In [ ]:
rmse = librosa.feature.rms(x, frame_length=frame_length, hop_length=hop_length, center=True)

print(rmse.shape)

rmse = rmse[0]

Plot da energia e RMSE em conjunto com a forma do sinal

In [ ]:
frames = range(len(energy))

t = librosa.frames_to_time(frames, sr=sr, hop_length=hop_length)

In [ ]:
plt.figure(figsize=(16, 6))

librosa.display.waveplot(x, sr=sr, alpha=0.4)
plt.plot(t, energy/energy.max(), 'r--')             # normalized for visualization
plt.plot(t[:len(rmse)], rmse/rmse.max(), color='g') # normalized for visualization
plt.legend(('Energy', 'RMSE'))
plt.show()

## Zero Crossing Rate
### Taxa de Travessia do Zero

A Taxa de Travessia do Zero do inglês [zero crossing rate](https://en.wikipedia.org/wiki/Zero-crossing_rate) (ZCR), é a taxa de alterações do sinal durante o determinado quadro de áudio. Em outras palavras, é o número de vezes que o sinal muda de valor, de positivo para negativo e vice-versa, dividido pelo comprimento do quadro. O ZCR é definido de acordo com a seguinte equação:

O ZCR pode ser interpretado como uma medida da invidade de um sinal. No geral, ele exibe valores mais elevados no caso de sinais barulhentos. Também é conhecido por refletir, de forma bastante grosseira, as características espectrais de um sinal. Tais propriedades, juntamente com o fato de ser fácil de calcular, levaram à sua adoção por inúmeras aplicações, incluindo detecção de fala e classificação de gênero musical, para citar apenas alguns.

> A ZCR é muito usada para detectar segmentos sonoros de curta duração. Ela assume que a energia está concentrada em baixas frequências para o sinal em questão, o que acarreta em poucas oscilações por unidade de tempo e, portanto, uma contagem baixa de passagens pelo zero. 

A Figura abaixo apresenta um sinal de fala junto com a respectiva sequência ZCR. Mostra que os valores do ZCR são mais elevados para as partes barulhentos do sinal, enquanto nos quadros de fala os respectivos valores ZCR são geralmente menores (dependendo, é claro, da natureza e contexto do fonema que é pronunciado a cada vez).

<div align="center" style="width: 100%;">
    <img src="imgs/zcr-in-voice.jpg">
</div>

In [ ]:
ipd.Audio(x, rate=sr)

Let's zoom in:

In [ ]:
n0 = 6500
n1 = 7500
plt.figure(figsize=(16, 6))
plt.plot(x[n0:n1])
plt.grid()
plt.show()

Vamos utilizar a librosa para obter a Taxa de Travessia do Zero do frame que estamos investigando. O resultado será o cálculo da máscara binária onde temos a presença de uma travessia do zero. Para encontrar o número total de travessias realizamos um sum.

In [ ]:
zero_crossings = librosa.zero_crossings(x[n0:n1], pad=False)

zero_crossings.shape

In [ ]:
print(sum(zero_crossings))

Para encontrar a ZCR ao longo do tempo, use a função zero_crossing_rate

In [ ]:
zcrs = librosa.feature.zero_crossing_rate(x)

print(zcrs.shape)

Plot do ZCR resultante:

In [ ]:
plt.figure(figsize=(16, 6))

plt.plot(zcrs[0])
plt.grid()
plt.show()

Observe como a alta taxa de cruzamento do zero corresponde à presença do snare drum. 

A razão para a alta taxa perto do início é porque o silêncio oscila discretamente em torno de zero:

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(x[:1000])
plt.ylim(-0.0001, 0.0001)
plt.show()

Um truque simples para contornar isso é adicionar uma pequena constante antes de calcular a RCS:

In [ ]:
zcrs = librosa.feature.zero_crossing_rate(x + 0.0001)
plt.figure(figsize=(14, 5))
plt.plot(zcrs[0])
plt.show()

## Spectral Features
### Características espectrais

Para classificação, vamos usar novas características em nosso arsenal: momentos espectrais (centroides, largura de banda, distorção, kurtose) e outras estatísticas espectrais.

Momentos ([*Moments*](https://en.wikipedia.org/wiki/Moment_(mathematics)) é um termo usado em física e estatística. Há momentos crus e momentos centrais.

Você provavelmente já está familiarizado com dois exemplos de momentos: média e variância. O primeiro momento bruto é conhecido como a média. O segundo momento central é conhecido como variância.

### Espectral

In [ ]:
x, sr = librosa.load('audios/simple_loop.wav')
ipd.Audio(x, rate=sr)

O centroide espectral (**spectral centroid** [Wikipedia](https://en.wikipedia.org/wiki/Spectral_centroid)) indica em que frequência a energia de um espectro é centrada. Isto é como uma média ponderada:

$$ f_c = \frac{\sum_k S(k) f(k)}{\sum_k S(k)} $$

Onde $S(k)$ é a magnitude espectral da frequência quando o bin for $k$, e $f(k)$ é a frequência do bin $k$.

[`librosa.feature.spectral_centroid`](https://librosa.github.io/librosa/generated/librosa.feature.spectral_centroid.html#librosa.feature.spectral_centroid) computa o centroide espectral para cada frame do sinal:

In [ ]:
spectral_centroids = librosa.feature.spectral_centroid(x, sr=sr)[0]
spectral_centroids.shape

In [ ]:
# Calcule a variável de tempo para visualização:

frames = range(len(spectral_centroids))
t = librosa.frames_to_time(frames)

Defina uma função para normalizar o centroide espectral para ajustar a visualização:

In [ ]:
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

Plot do centroide espectral junto com a forma do sinal:

In [ ]:
plt.figure(figsize=(16, 4))
librosa.display.waveplot(x, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_centroids), color='r') # normalize for visualization purposes
plt.show()

Semelhante à taxa de travessia zero, há um aumento espúrio no centroide espectral no início do sinal. Isso porque o silêncio no início tem uma amplitude tão pequena que os componentes de alta frequência têm a chance de dominar. Um hack em torno disso é adicionar uma pequena constante antes de calcular o centroide espectral, mudando assim o centroide para zero em porções tranquilas:

In [ ]:
plt.figure(figsize=(16, 4))
spectral_centroids = librosa.feature.spectral_centroid(x+0.01, sr=sr)[0]
librosa.display.waveplot(x, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_centroids), color='r') # normalize for visualization purposes
plt.show()

## Fourier Transform
### Transformada de Fourier

A Transformada de Fourier ([Wikipedia](https://en.wikipedia.org/wiki/Fourier_transform)) é uma das operações mais fundamentais no processamento de sinal e matemática aplicada.

Ele transforma nosso sinal no domínio do tempo para o domínio da frequência. Enquanto o domínio do tempo expressa nosso sinal como uma sequência de amostras, o domínio da frequência expressa nosso sinal como uma superposição de sinusóides de magnitudes, frequências e compensações de fase variáveis.

Para calcular a Transformada de Fourier em NumPy ou SciPy, use [scipy.fft](http://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.fft.html#scipy.fftpack.fft):

In [ ]:
x, sr = librosa.load('audios/c_strum.wav')

In [ ]:
print(x.shape) 
print(sr)

In [ ]:
ipd.Audio(x, rate=sr)

In [ ]:
X = scipy.fft(x)
X_mag = np.absolute(X)
f = np.linspace(0, sr, len(X_mag)) # frequency variable

Plot do espectro:

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(f, X_mag) # magnitude spectrum
plt.xlabel('Frequency (Hz)')
plt.show()

### Zoom in:

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(f[:5000], X_mag[:5000])
plt.xlabel('Frequency (Hz)')
plt.show()

## Short-Time Fourier Transform
### Transformada de fourier de curto prazo

Os sinais musicais são altamente não estacionários, ou seja, suas estatísticas mudam ao longo do tempo. Seria um tanto sem sentido calcular uma única transformação de Fourier ao longo de uma música inteira de 10 minutos.

A transformada de Fourier de curta duração (STFT) (Wikipedia; FMP, p. 53) é obtida calculando a transformada de Fourier para quadros sucessivos em um sinal.

$$\Large X(m, \omega) = \sum_n x(n) w(n-m) e^{-j \omega n} $$

À medida que aumentamos  $\large m$, nós deslizamos a janela da função $\large w$ à direita. Para o quadro resultante, $\large x(n) w(n-m)$, nós calculamos a transformada de Fourier. Logo, o STFT $\large X$ é uma função tanto para o tempo ($\large m$), quanto a frequência ($\large \omega$).

In [ ]:
x, sr = librosa.load('audios/brahms_hungarian_dance_5.mp3')

ipd.Audio(x, rate=sr)

Utilize o [librosa.stft](https://librosa.github.io/librosa/generated/librosa.core.stft.html#librosa.core.stft) para computar o STFT.

Nós informamos o frame size, ou seja, o tamanho da FFT, e um hop length, ou seja, o valor que deve ser incrementado ao frame size:

In [ ]:
hop_length = 512
n_fft = 2048
X = librosa.stft(x, n_fft=n_fft, hop_length=hop_length)

Converte o **comprimento do salto** (hop length) e o tamanho do quadro (frame size) em segundos:

In [ ]:
float(hop_length)/sr # units of seconds

In [ ]:
float(n_fft)/sr  # units of seconds

Para sinais reais, a transformação de Fourier é simétrica sobre o ponto médio. Portanto, só retém metade da saída: librosa.stft

In [ ]:
X.shape

Em nosso exemplo, o STFT gerou 1025 frequency bins e 9813 frames.

## Spectrogram
### Espectrograma

No processamento de áudio, muitas vezes só nos preocupamos com a magnitude espectral e não com o conteúdo de fase em análise.

O Espectrograma ([Wikipedia](https://en.wikipedia.org/wiki/Spectrogram); FMP, p. 29, 55) mostra a intensidade das frequências ao longo do tempo. Um espectrograma é simplesmente a magnitude quadrada do STFT:

$$\Large S(m, \omega) = \left| X(m, \omega) \right|^2$$

A percepção humana da intensidade sonora é logarítmica na natureza. Portanto, muitas vezes estamos interessados na amplitude do áudio:

In [ ]:
S = librosa.amplitude_to_db(abs(X))

Para exibir o espectrograma via librosa: [librosa.display.specshow](http://bmcfee.github.io/librosa/generated/librosa.display.specshow.html).

In [ ]:
plt.figure(figsize=(20, 5))
librosa.display.specshow(S, sr=sr, hop_length=hop_length, x_axis='time', y_axis='linear')
plt.colorbar(format='%+2.0f dB')
plt.show()

## Mel-spectrogram

[librosa.feature.melspectrogram](http://bmcfee.github.io/librosa/generated/librosa.feature.melspectrogram.html#librosa.feature.melspectrogram):

In [ ]:
hop_length = 256
S = librosa.feature.melspectrogram(x, sr=sr, n_fft=4096, hop_length=hop_length)

A percepção humana da intensidade sonora é logarítmica por natureza. Portanto, como o espectrograma baseado no STFT, muitas vezes estamos interessados na amplitude log:

In [ ]:
logS = librosa.power_to_db(abs(S))

In [ ]:
plt.figure(figsize=(20, 5))
librosa.display.specshow(logS, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.show()

[Mel scale](https://en.wikipedia.org/wiki/Mel_scale) é similar a função $\log (1 + f)$:

$$m = 2595 \log_{10} \left(1 + \frac{f}{700} \right)$$

# Autocorrelation
### Autocorrelação

A autocorrelação ([autocorrelation](http://en.wikipedia.org/wiki/Autocorrelation)) de um sinal descreve a semelhança de um sinal contra uma versão diferenciada de tempo de si mesmo. Para um sinal $x$, a autocorrelação $r$ é:

$$ r(k) = \sum_n x(n) x(n-k) $$, onde $k$ é frequentemente chamado de parâmetro **lag**. 

The autocorrelation is useful for finding repeated patterns in a signal. For example, at short lags, the autocorrelation can tell us something about the signal's fundamental frequency. For longer lags, the autocorrelation may tell us something about the tempo of a musical signal.

A autocorrelação é útil para encontrar padrões repetidos em um sinal. Por exemplo, em problemas de defasagem curta, a autocorrelação pode nos dizer algo sobre a frequência fundamental do sinal. Para atrasos mais longos, a autocorrelação pode nos dizer coisas como informações sobre o ritmo de um sinal musical.

In [ ]:
x, sr = librosa.load('audios/c_strum.wav')
ipd.Audio(x, rate=sr)

In [ ]:
plt.figure(figsize=(14, 5))
librosa.display.waveplot(x, sr)
plt.show()

### `numpy.correlate`

Há duas maneiras de calcular a autocorrelação em Python. O primeiro é o método [`numpy.correlate`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.correlate.html):

In [ ]:
# Because the autocorrelation produces a symmetric signal, we only care about the "right half".

r = np.correlate(x, x, mode='full')[len(x)-1:]
print(x.shape, r.shape)

In [ ]:
# Plot the autocorrelation:

plt.figure(figsize=(14, 5))
plt.plot(r[:10000])
plt.xlabel('Lag (samples)')
plt.xlim(0, 10000)
plt.show()

### `librosa.autocorrelate`

O segundo método é utilizar diretamente o [`librosa.autocorrelate`](http://bmcfee.github.io/librosa/generated/librosa.core.autocorrelate.html#librosa.core.autocorrelate):

In [ ]:
r = librosa.autocorrelate(x, max_size=10000)
print(r.shape)

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(r)
plt.xlabel('Lag (samples)')
plt.xlim(0, 10000)
plt.show()

`librosa.autocorrelate` convenientemente só mantém metade da função de autocorrelação, uma vez que a função de autocorrelação é simétrica. Além disso, a utilização do parâmetro `max_size` evita cálculos.

## Pitch Estimation

A autocorrelação é usada para encontrar padrões repetidos dentro de um sinal. Para sinais musicais, um padrão repetido pode corresponder a um período de pitch, ou no geral, o tom da música. 

Podemos, portanto, usar a função de autocorrelação para estimar o tom em um sinal musical.

In [ ]:
x, sr = librosa.load('audios/oboe_c6.wav')
ipd.Audio(x, rate=sr)

In [ ]:
# Compute and plot the autocorrelation:

r = librosa.autocorrelate(x, max_size=5000)
plt.figure(figsize=(14, 5))
plt.plot(r[:200])
plt.show()

The autocorrelation always has a maximum at zero, i.e. zero lag. We want to identify the maximum outside of the peak centered at zero. Therefore, we might choose only to search within a range of reasonable pitches:

A autocorrelação sempre tem um máximo de zero, ou seja, zero lag. Queremos identificar o máximo fora do pico centrado em zero. Portanto, podemos escolher apenas pesquisar dentro de uma gama de arremessos razoáveis:

In [ ]:
midi_hi = 120.0
midi_lo = 12.0
f_hi = librosa.midi_to_hz(midi_hi)
f_lo = librosa.midi_to_hz(midi_lo)
t_lo = sr/f_hi
t_hi = sr/f_lo

In [ ]:
print(f_lo, f_hi)
print(t_lo, t_hi)

Definir os pontos/tons inválidos:

In [ ]:
r[:int(t_lo)] = 0
r[int(t_hi):] = 0

plt.figure(figsize=(14, 5))
plt.plot(r[:1400])
plt.show()

In [ ]:
# Encontra o ponto de valor máximo:

t_max = r.argmax()
print(t_max)

Neste ponto, vamos estimar o **tom** em Hertz:

In [ ]:
float(sr)/t_max

Na verdade, isso é muito próximo da frequência real de um **C6**:

In [ ]:
librosa.midi_to_hz(84)

<div align="center" style="width: 100%;">
    <img src="imgs/piano-scale-hertz-frequency-notes.png">
</div>

<div align="center" style="width: 100%;">
    <img src="http://newt.phys.unsw.edu.au/jw/graphics/notes.GIF">
   
    ref: http://newt.phys.unsw.edu.au/jw/notes.html
</div>

# Genre Recognition

Carregando 30 segundos para verificação/exploração.

In [ ]:
filename_brahms = 'audios/brahms_hungarian_dance_5.mp3'

x_brahms, sr_brahms = librosa.load(filename_brahms, duration=30)

In [ ]:
filename_busta = 'audios/busta_rhymes_hits_for_days.mp3'

x_busta, sr_busta = librosa.load(filename_busta, duration=30)

In [ ]:
ipd.Audio(x_brahms, rate=sr_brahms)

In [ ]:
ipd.Audio(x_busta, rate=sr_busta)

Exibir o formato da onda no domínio do tempo dos sinais de áudio em análise:

In [ ]:
plt.figure(figsize=(18, 4))
librosa.display.waveplot(x_brahms, sr_brahms)
plt.show()

In [ ]:
plt.figure(figsize=(18, 4))
librosa.display.waveplot(x_busta, sr_busta)
plt.show()

Computar o melspectrogram de potência:

In [ ]:
S_brahms = librosa.feature.melspectrogram(x_brahms, sr=sr_brahms, power=2.0)
S_busta = librosa.feature.melspectrogram(x_busta, sr=sr_busta, power=2.0)

Converter amplitude em decibéis:

In [ ]:
Sdb_brahms = librosa.power_to_db(S_brahms)
Sdb_busta = librosa.power_to_db(S_busta)

In [ ]:
plt.figure(figsize=(18, 6))
librosa.display.specshow(Sdb_brahms, sr=sr_brahms, x_axis='time', y_axis='mel')
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(18, 6))
librosa.display.specshow(Sdb_busta, sr=sr_busta, x_axis='time', y_axis='mel')
plt.colorbar()
plt.show()